<a id="indice"></a> 
## Índice de la notebook

[1) PYTORCH AGAINS PYTORCH LIGHTNING](#1)<BR>

[2) TURNING A PYTORCH CODE FROM intro_pytorch to PYTORCH LIGHTNING](#2)<BR>

[3) INTRODUCTION TO PYTORCH LIGHTNING](#3)
-    [3.1)ADDING VALIDATE AND TEST STEPS](#4)<BR>
-    [3.2)SAVING AND LOADING CHECKPOINTS](#5)<BR>
-    [3.3)EARLY STOPPING (REGULARIZATION)](#6)<BR>
-    [3.4)DEBUG YOUR MODEL](#7)<BR>
-    [3.5)TRACK AND VISUALIZATION](#8)<BR>
-    [3.6)DEPLOY MODELS INTO PRODUCCTION](#9)<BR>


<a id="1"></a>
## 1)PYTORCH AGAINS PYTORCH LIGHTNING 

[Ir a índice](#indice)

#### Some details that we dont need to worry about with pyTorch Lightening

## <u>setting</u>
model.eval()
model.train()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
data.to(device)
#### -> easy GPU/TPU support
#### -> scale GPUs

optimizer.zero_grad()
loss.backward()
optimizer.step()

with torch.no_grad():
    ...

x = x.detach()

#### Bonus : - Tensorboard support
####         - prints tips/hints

<a id="2"></a> 

### 2) TURNING A PYTORCH CODE FROM intro_pytorch to PYTORCH LIGHTNING

[Ir a índice](#indice)

*Here i transform from PyTorch to PyTorch Lightning a code which aims to predict the labels of handwritten digits from MNIST dataset*

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import pytorch_lightning as pl
from pytorch_lightning import Trainer


# Hyper-parameters
input_size = 784 # 28x28
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 1e-3

class LightNeuralNet(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LightNeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out
    
    def training_step(self, batch, batch_idx):
        X_train, y_train = batch
        X_train = X_train.reshape(-1, 28*28)
        
        # forward pass
        y_hat = self(X_train)
        loss = F.cross_entropy(y_hat, y_train)
        
        # Shows in Tensorboard
        self.log('train_loss', loss, on_epoch=False)
        return loss
    
    def train_dataloader(self):
        train_dataset = torchvision.datasets.MNIST(root = './intro_pytorch_data', 
                                                   train = True, download = False, 
                                                   transform = transforms.ToTensor())
        
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, 
                                                   num_workers = 8, shuffle = True)
        return train_loader
    
    def val_dataloader(self):
        val_dataset = torchvision.datasets.MNIST(root = './intro_pytorch_data', 
                                                 train = False, download = False, 
                                                 transform = transforms.ToTensor())
        
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 32, num_workers = 8, shuffle = False)
        return val_loader
    
    
    ################# VALIDATION STEP #################
    def validation_step(self, batch, batch_idx):
        X_val, y_val = batch
        X_val = X_val.reshape(-1, 28*28)
        
        # forward pass
        y_hat = self(X_val)
        
        loss = F.cross_entropy(y_hat, y_val)
        self.log('avg_val_loss', loss)#pytorch Lightning automatically mean accumulates the metric and averages by epoch
        return loss



    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = learning_rate)


    
if __name__ == "__main__":
    model = LightNeuralNet(input_size, hidden_size, num_classes)
    trainer = Trainer(max_epochs = num_epochs, # gpus=1 is the number of gpu used//tpu_cores is the number of cores
                      fast_dev_run = True, deterministic=True) # auto_lr_rate = True, find the best lr
    trainer.fit(model)

To open Tensorboard: <br>
`tensorboard --logdir=lightning_logs`

<a id="3"></a> 
### 3) INTRODUCTION TO PYTORCH LIGHTNING<br>


[Ir a índice](#indice)
##### Here i run the introductory guide from PyTorch Lightning website implementing an Vanilla Autoencoder with pytorch<br> 

https://pytorch-lightning.readthedocs.io/en/stable/levels/core_skills.html

In [12]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import pytorch_lightning as pl

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=3)
        )        
    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=3, out_features= 64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=28*28)
        )
    def forward(self, x):
        return self.l1(x)

# Defining the lightling module
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, Encoder, Decoder):
        super().__init__()
        self.encoder = Encoder
        self.decoder = Decoder
        
    def training_step(self, batch, batch_idx):
        # training loop
        X, y = batch
        X = X.view(X.size(0), -1)        
        z = self.encoder(X)
        X_hat = self.decoder(z)
        loss = F.mse_loss(X_hat, X)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer
    
# Defining the training dataset
train_dataset = MNIST(root = './intro_pytorch_data', train = True, download = False, 
                        transform = transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, shuffle= True, num_workers=8)
        
# instance the model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train the model using the lightining trainer
trainer = pl.Trainer()
trainer.fit(model= autoencoder, train_dataloaders= train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/csuarezgurruchaga/opt/anaconda3/envs/redes/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 50.4 K
1 | decoder | Decoder | 51.2 K
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Epoch 0:   2%|▏         | 1139/60000 [00:11<10:02, 97.68it/s, loss=0.0502, v_num=2]

<a id="4"></a> 
### 3.1)ADDING VALIDATE AND TEST STEPS<br>


[Ir a índice](#indice)

https://pytorch-lightning.readthedocs.io/en/stable/common/evaluation_basic.html

In [13]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=3)
        )        
    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=3, out_features= 64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=28*28)
        )
    def forward(self, x):
        return self.l1(x)

# Defining the lightling module
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, Encoder, Decoder):
        super().__init__()
        self.encoder = Encoder
        self.decoder = Decoder
        
    def training_step(self, batch, batch_idx):
        # training loop
        X, y = batch
        X = X.view(X.size(0), -1)        
        z = self.encoder(X)
        X_hat = self.decoder(z)
        loss = F.mse_loss(X_hat, X)
        return loss
    
    #Define the validation loop
    def validation_step(self, batch, batch_idx):
        X, y = batch
        X = X.view(X.size(0), -1)
        z = self.encoder(X)
        X_hat = self.decoder(z)
        val_loss = F.mse_loss(X_hat, X)
        self.log("val_loss", val_loss)
        return val_loss
        
    #Define the test loop
    def test_step(self, batch, batch_idx):
        X, y = batch
        X = X.view(X.size(0), -1)
        z = self.encoder(X)
        X_hat = self.decoder(z)
        test_loss = F.mse_loss(X_hat, X)
        self.log("test_loss", test_loss)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer
    
# Defining the training dataset
transform = transforms.ToTensor()

#### ADDING VALIDATION LOOP ####
train_dataset = MNIST(root = './intro_pytorch_data', train = True, download = False, 
                        transform = transform)
# use 20% of training data for validation
train_set_size = int(len(train_dataset) * 0.8)
valid_set_size = len(train_dataset) - train_set_size

# split the train set into two
seed = torch.Generator().manual_seed(42)
train_dataset, valid_dataset = random_split(train_dataset, [train_set_size, valid_set_size], generator=seed)

train_loader = DataLoader(dataset=train_dataset, shuffle= True, num_workers=8)
val_loader = DataLoader(dataset=valid_dataset, shuffle= False, num_workers=8)
        
        
        
        
test_dataset = MNIST(root = './intro_pytorch_data', train = False, download = False, 
                        transform = transform)
test_loader = DataLoader(dataset=test_dataset, shuffle= False, num_workers=8)
# instance the model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train the model using the lightining trainer FOR TRAIN/VAL AND TEST DATASETS
trainer = pl.Trainer(max_epochs = 2, fast_dev_run = True)
trainer.fit(model= autoencoder, train_dataloaders= train_loader, 
                                val_dataloaders=val_loader)


#### Train with the test loop ####
trainer.test(model = autoencoder, dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
/Users/csuarezgurruchaga/opt/anaconda3/envs/redes/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /Users/csuarezgurruchaga/Desktop/UDESA/008_redes_neuronales/01_trabajos_practicos/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 50.4 K
1 | decoder | Decoder | 51.2 K
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
/Users/csuarezgurruchaga/opt/anaconda3/envs/redes/lib/python3.8/site-packa

Epoch 0: 100%|██████████| 2/2 [00:06<00:00,  3.22s/it, loss=0.142, v_num=]

`Trainer.fit` stopped: `max_steps=1` reached.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 122.53it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.09412974864244461
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.09412974864244461}]

<a id="5"></a> 
### 3.2)SAVING AND LOADING CHECKPOINTS<br>
[Ir a índice](#indice)


https://pytorch-lightning.readthedocs.io/en/stable/common/checkpointing_basic.html#save-a-checkpoint

- simply by using the Trainer you get automatic checkpointing

In [18]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=3)
        )        
    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=3, out_features= 64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=28*28)
        )
    def forward(self, x):
        return self.l1(x)

# Defining the lightling module
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, Encoder, Decoder):
        super().__init__()
        self.encoder = Encoder
        self.decoder = Decoder
        self.save_hyperparameters() # Saving the hiperparameters pased to the init
        
    def training_step(self, batch, batch_idx):
        # training loop
        X, y = batch
        X = X.view(X.size(0), -1)        
        z = self.encoder(X)
        X_hat = self.decoder(z)
        loss = F.mse_loss(X_hat, X)
        return loss
    
    #Define the validation loop
    def validation_step(self, batch, batch_idx):
        X, y = batch
        X = X.view(X.size(0), -1)
        z = self.encoder(X)
        X_hat = self.decoder(z)
        val_loss = F.mse_loss(X_hat, X)
        self.log("val_loss", val_loss)
        return val_loss
        
    #Define the test loop
    def test_step(self, batch, batch_idx):
        X, y = batch
        X = X.view(X.size(0), -1)
        z = self.encoder(X)
        X_hat = self.decoder(z)
        test_loss = F.mse_loss(X_hat, X)
        self.log("test_loss", test_loss)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer
    
# Defining the training dataset
transform = transforms.ToTensor()

#### ADDING VALIDATION LOOP ####
train_dataset = MNIST(root = './intro_pytorch_data', train = True, download = False, 
                        transform = transform)
# use 20% of training data for validation
train_set_size = int(len(train_dataset) * 0.8)
valid_set_size = len(train_dataset) - train_set_size

# split the train set into two
seed = torch.Generator().manual_seed(42)
train_dataset, valid_dataset = random_split(train_dataset, [train_set_size, valid_set_size], generator=seed)

train_loader = DataLoader(dataset=train_dataset, shuffle= True, num_workers=8)
val_loader = DataLoader(dataset=valid_dataset, shuffle= False, num_workers=8)
        
        
        
        
test_dataset = MNIST(root = './intro_pytorch_data', train = False, download = False, 
                        transform = transform)
test_loader = DataLoader(dataset=test_dataset, shuffle= False, num_workers=8)
# instance the model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train the model using the lightining trainer FOR TRAIN/VAL AND TEST DATASETS
# saves checkpoints to 'some/path/' at every epoch end
trainer = pl.Trainer(max_epochs = 2, fast_dev_run = False, default_root_dir= "./checkpoints")
trainer.fit(model= autoencoder, train_dataloaders= train_loader, 
                                val_dataloaders=val_loader)


#### Train with the test loop ####
trainer.test(model = autoencoder, dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: checkpoints/lightning_logs

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 50.4 K
1 | decoder | Decoder | 51.2 K
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Epoch 1: 100%|██████████| 60000/60000 [06:10<00:00, 162.12it/s, loss=0.0374, v_num=0]

`Trainer.fit` stopped: `max_epochs=2` reached.


Testing DataLoader 0: 100%|██████████| 10000/10000 [00:23<00:00, 422.96it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.04018773138523102
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.04018773138523102}]

In [ ]:
#### Loading a saved checkpoint ####
model = pl.LightningModule.load_from_checkpoint("checkpoints/epoch=0-step=48000.ckpt")
print(model.learning_rate) # also if you set self.save_hyperparameters(), you can call for the hiperparameters saved.
# disable randomness, dropout, etc...
model.eval()

# predict with the model
y_hat = model(X)

#### For resuming training stage ####
model = LitAutoEncoder(Encoder, Decoder)
trainer = Trainer()

# automatically restores model, epoch, step, LR schedulers, etc...
trainer.fit(model, ckpt_path="some/path/to/my_checkpoint.ckpt")

<a id="6"></a> 
### 3.3) EARLY STOPPING (REGULARIZATION)<br>
[Ir a índice](#indice)

https://pytorch-lightning.readthedocs.io/en/stable/common/early_stopping.html

In [20]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=3)
        )        
    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=3, out_features= 64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=28*28)
        )
    def forward(self, x):
        return self.l1(x)

# Defining the lightling module
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, Encoder, Decoder):
        super().__init__()
        self.encoder = Encoder
        self.decoder = Decoder
        
    def training_step(self, batch, batch_idx):
        # training loop
        X, y = batch
        X = X.view(X.size(0), -1)        
        z = self.encoder(X)
        X_hat = self.decoder(z)
        loss = F.mse_loss(X_hat, X)
        return loss
    
    #Define the validation loop
    def validation_step(self, batch, batch_idx):
        X, y = batch
        X = X.view(X.size(0), -1)
        z = self.encoder(X)
        X_hat = self.decoder(z)
        val_loss = F.mse_loss(X_hat, X)
        self.log("val_loss", val_loss)
        return val_loss
        
    #Define the test loop
    def test_step(self, batch, batch_idx):
        X, y = batch
        X = X.view(X.size(0), -1)
        z = self.encoder(X)
        X_hat = self.decoder(z)
        test_loss = F.mse_loss(X_hat, X)
        self.log("test_loss", test_loss)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer
    
# Defining the training dataset
transform = transforms.ToTensor()

#### ADDING VALIDATION LOOP ####
train_dataset = MNIST(root = './intro_pytorch_data', train = True, download = False, 
                        transform = transform)
# use 20% of training data for validation
train_set_size = int(len(train_dataset) * 0.8)
valid_set_size = len(train_dataset) - train_set_size

# split the train set into two
seed = torch.Generator().manual_seed(42)
train_dataset, valid_dataset = random_split(train_dataset, [train_set_size, valid_set_size], generator=seed)

train_loader = DataLoader(dataset=train_dataset, shuffle= True, num_workers=8)
val_loader = DataLoader(dataset=valid_dataset, shuffle= False, num_workers=8)
        
        
        
        
test_dataset = MNIST(root = './intro_pytorch_data', train = False, download = False, 
                        transform = transform)
test_loader = DataLoader(dataset=test_dataset, shuffle= False, num_workers=8)
# instance the model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train the model using the lightining trainer FOR TRAIN/VAL AND TEST DATASETS
trainer = pl.Trainer(max_epochs = 2, callbacks=[EarlyStopping(monitor="val_loss", mode = 'min')])

trainer.fit(model= autoencoder, train_dataloaders= train_loader, 
                                val_dataloaders=val_loader)


#### Train with the test loop ####
trainer.test(model = autoencoder, dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 50.4 K
1 | decoder | Decoder | 51.2 K
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Epoch 1: 100%|██████████| 60000/60000 [06:08<00:00, 162.61it/s, loss=0.0458, v_num=19]

`Trainer.fit` stopped: `max_epochs=2` reached.


Testing DataLoader 0: 100%|██████████| 10000/10000 [00:22<00:00, 438.18it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.04136969521641731
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.04136969521641731}]

<a id="7"></a> 
### 3.4) DEBUG YOUR MODEL<br>
[Ir a índice](#indice)

https://pytorch-lightning.readthedocs.io/en/stable/debug/debugging_basic.html

<a id="8"></a> 
### 3.5) TRACK AND VISUALIZATION<br>
[Ir a índice](#indice)

In model development, we track values of interest such as the validation_loss to visualize the learning process for our models. Model development is like driving a car without windows, charts and logs provide the windows to know where to drive the car.<BR>

https://pytorch-lightning.readthedocs.io/en/stable/visualize/logging_basic.html

In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=3)
        )        
    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features=3, out_features= 64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=28*28)
        )
    def forward(self, x):
        return self.l1(x)

# Defining the lightling module
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, Encoder, Decoder, batch_size): # we set the "batch_size" parameter, to use auto_scale_batch_size
        super().__init__()
        self.encoder = Encoder
        self.decoder = Decoder
        
    def training_step(self, batch, batch_idx):
        # training loop
        X, y = batch
        X = X.view(X.size(0), -1)        
        z = self.encoder(X)
        X_hat = self.decoder(z)
        loss = F.mse_loss(X_hat, X)
        return loss
    
    #Define the validation loop
    def validation_step(self, batch, batch_idx):
        X, y = batch
        X = X.view(X.size(0), -1)
        z = self.encoder(X)
        X_hat = self.decoder(z)
        val_loss = F.mse_loss(X_hat, X)
        self.log("val_loss", val_loss)
        return val_loss
        
    #Define the test loop
    def test_step(self, batch, batch_idx):
        X, y = batch
        X = X.view(X.size(0), -1)
        z = self.encoder(X)
        X_hat = self.decoder(z)
        test_loss = F.mse_loss(X_hat, X)
        self.log("test_loss", test_loss, prog_bar = True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer
    
# Defining the training dataset
transform = transforms.ToTensor()

#### ADDING VALIDATION LOOP ####
train_dataset = MNIST(root = './intro_pytorch_data', train = True, download = False, 
                        transform = transform)
# use 20% of training data for validation
train_set_size = int(len(train_dataset) * 0.8)
valid_set_size = len(train_dataset) - train_set_size

# split the train set into two
seed = torch.Generator().manual_seed(42)
train_dataset, valid_dataset = random_split(train_dataset, [train_set_size, valid_set_size], generator=seed)

train_loader = DataLoader(dataset=train_dataset, shuffle= True, num_workers=8)
val_loader = DataLoader(dataset=valid_dataset, shuffle= False, num_workers=8)
        
        
        
        
test_dataset = MNIST(root = './intro_pytorch_data', train = False, download = False, 
                        transform = transform)
test_loader = DataLoader(dataset=test_dataset, shuffle= False, num_workers=8)
# instance the model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train the model using the lightining trainer FOR TRAIN/VAL AND TEST DATASETS
trainer = pl.Trainer(max_epochs = 2, auto_scale_batch_size= True, auto_lr_find= True) # Finds the better batch size and lr when we run trainer.tune(model)

# trainer.tune(model) # Runs routines to find the better hiperparameters before training, we also need set 'auto_lr_find' in trainer 

trainer.fit(model= autoencoder, train_dataloaders= train_loader, val_dataloaders=val_loader)

#### Train with the test loop ####
trainer.test(model = autoencoder, dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 50.4 K
1 | decoder | Decoder | 51.2 K
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Epoch 1: 100%|██████████| 60000/60000 [05:40<00:00, 176.06it/s, loss=0.0416, v_num=21]

`Trainer.fit` stopped: `max_epochs=2` reached.


Testing DataLoader 0: 100%|██████████| 10000/10000 [00:23<00:00, 421.62it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.040752630680799484
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.040752630680799484}]

<a id="9"></a> 
### 3.6)DEPLOY MODELS INTO PRODUCTION<BR>
[Ir a índice](#indice)


https://pytorch-lightning.readthedocs.io/en/stable/deploy/production_basic.html

In [14]:
# Converting a PyTorch code from intro_pytorch to PyTorch Lightening

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import pytorch_lightning as pl
from pytorch_lightning import Trainer


# Hyper-parameters
input_size = 784 # 28x28
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 1e-3

class LightNeuralNet(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LightNeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out
    
    def training_step(self, batch, batch_idx):
        X_train, y_train = batch
        X_train = X_train.reshape(-1, 28*28)
        
        # forward pass
        y_hat = self(X_train)
        loss = F.cross_entropy(y_hat, y_train)
        
        # Shows in Tensorboard
        self.log('train_loss', loss, on_epoch=False)
        return loss
    
    def train_dataloader(self):
        train_dataset = torchvision.datasets.MNIST(root = './intro_pytorch_data', 
                                                   train = True, download = False, 
                                                   transform = transforms.ToTensor())
        
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, 
                                                   num_workers = 8, shuffle = True)
        return train_loader
    
    def val_dataloader(self):
        val_dataset = torchvision.datasets.MNIST(root = './intro_pytorch_data', 
                                                 train = False, download = False, 
                                                 transform = transforms.ToTensor())
        
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 32, num_workers = 8, shuffle = False)
        return val_loader
    
    
    ################# VALIDATION STEP #################
    def validation_step(self, batch, batch_idx):
        X_val, y_val = batch
        X_val = X_val.reshape(-1, 28*28)
        
        # forward pass
        y_hat = self(X_val)
        
        loss = F.cross_entropy(y_hat, y_val)
        self.log('avg_val_loss', loss)#pytorch Lightning automatically mean accumulates the metric and averages by epoch
        return loss



    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = learning_rate)


    def predict_step(self, batch, batch_idx):
        output = self(batch) 
        return output 

    
if __name__ == "__main__":
    model = LightNeuralNet(input_size, hidden_size, num_classes)
    trainer = Trainer(max_epochs = num_epochs, # gpus=1 is the number of gpu used//tpu_cores is the number of cores
                      fast_dev_run = True, deterministic=True) # auto_lr_rate = True, find the best lr
    trainer.fit(model)
    
    

test_data = torch.rand(784,784)
trainer.predict(model, test_data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 392 K 
1 | relu | ReLU   | 0     
2 | l2   | Linear | 5.0 K 
--------------------------------
397 K     Trainable params
0         Non-trainable params
397 K     Total params
1.590     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 2/2 [00:11<00:00,  5.88s/it, loss=2.32, v_num=]

`Trainer.fit` stopped: `max_steps=1` reached.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 82.07it/s] 


[tensor([-0.1411, -0.1731,  0.2701,  0.5368,  0.0308,  0.2903,  0.1149, -0.0087,
         -0.1700, -0.0348])]